### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/1

In [28]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages. Youu can also include links to social media pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"},
        {"type": "social media page", "url": "https://another.full.url/social-media"}
    ]
}
"""

In [29]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [30]:
print(get_links_user_prompt("https://huggingface.co"))


Here is the list of links on the website https://huggingface.co -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

/
/models
/datasets
/spaces
/docs
/enterprise
/pricing
/login
/join
/spaces
/models
/zai-org/GLM-Image
/fal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA
/Lightricks/LTX-2
/openbmb/AgentCPM-Explore
/google/translategemma-4b-it
/models
/spaces/multimodalart/qwen-image-multiple-angles-3d-camera
/spaces/mrfakename/Z-Image-Turbo
/spaces/Wan-AI/Wan2.2-Animate
/spaces/prithivMLmods/Qwen-Image-Edit-2511-LoRAs-Fast
/spaces/r3gm/wan2-2-fp8da-aoti-preview
/spaces
/datasets/MiniMaxAI/OctoCodingBench
/datasets/HuggingFaceFW/finetranslations
/datasets/Alibaba-Apsara/Superior-Reasoning-SFT-gpt-oss-120b
/datasets/miromind-ai/MiroVerse-v0.1
/datasets/ScienceOne-AI/S1-MMAlign
/datasets
/join
/enterprise
/enterprise

In [31]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [32]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 6 relevant links


{'links': [{'type': 'company page', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'social media page', 'url': 'https://github.com/huggingface'},
  {'type': 'social media page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'social media page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

In [33]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        if link['type'] == 'social media page':
            print(f"Skipping social media page: {link['url']}");
            result += f"\n\n### Social media Link: {link['url']}\n"
            continue
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result
    

In [34]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links
Skipping social media page: https://github.com/huggingface
Skipping social media page: https://twitter.com/huggingface
Skipping social media page: https://www.linkedin.com/company/huggingface/
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
zai-org/GLM-Image
Updated
3 days ago
•
6k
•
803
fal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA
Updated
11 days ago
•
51k
•
734
Lightricks/LTX-2
Updated
4 days ago
•
1.46M
•
1.13k
openbmb/AgentCPM-Explore
Updated
4 days ago
•
1.41k
•
323
google/translategemma-4b-it
Updated
3 days ago
•
5.38k
•
280
Browse 2M+ models
Spaces
Running
on
Zero
Featured

In [35]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information. If you find social media links, include them as hyperlinks in the markdown.
"""

# Or Different 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

In [20]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nzai-org/GLM-Image\nUpdated\n3 days ago\n•\n6k\n•\n801\nfal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA\nUpdated\n11 days ago\n•\n51k\n•\n734\nLightricks/LTX-2\nUpdated\n4 days ago\n•\n1.46M\n•\n1.13k\nopenbmb/AgentCPM-Explore\nUpdated\n4 days ago\n•\n1.41k\n•\n323\ngoogle/translategemma-4b-it\nUpdated\n3 days ago\n•\n5.38k\n•\n280\nBrowse 2M+ models\nSpaces\nRunning\non\nZero\nFeatured\n891\n

In [25]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [37]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [38]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 6 relevant links
Skipping social media page: https://github.com/huggingface
Skipping social media page: https://twitter.com/huggingface
Skipping social media page: https://www.linkedin.com/company/huggingface/
Skipping social media page: https://discuss.huggingface.co


# Hugging Face Brochure

---

## Who We Are

Hugging Face is the AI community building the future of machine learning. As a central collaboration platform, we empower machine learning engineers, scientists, and end users worldwide to share, explore, and develop open-source models, datasets, and applications. Our mission is to create an open and ethical AI future through community-driven innovation.

---

## Our Platform

- **Models:** Browse over 2 million machine learning models in diverse modalities including text, image, video, audio, and 3D.
- **Datasets:** Access more than 500,000 datasets for training and benchmarking AI models.
- **Spaces:** Explore and build AI-powered applications hosted and shared on our platform.
- **Collaboration:** Host and contribute to unlimited public projects with a fast-growing supportive community.
- **Compute & Enterprise:** Accelerate your machine learning projects with paid compute solutions and enterprise-grade tools.

---

## Our Community & Culture

We foster an inclusive and talented community at the forefront of AI innovation, promoting open-source collaboration and ethical development. Our culture centers around:

- Empowering learning and growth for machine learning engineers and scientists.
- Encouraging portfolio building and sharing work openly.
- Open communication through active forums and community spaces.
- Emphasizing transparency, ethics, and teamwork in AI research and deployment.

---

## Careers at Hugging Face

Join a passionate and innovative team driving the AI revolution. We are continuously expanding and offer opportunities across research, engineering, product, and more. Be part of a company that values collaboration, learning, and impact.

Explore current openings and learn more about our career opportunities on our [Careers Page](https://huggingface.co/careers).

---

## Our Customers & Impact

Hugging Face supports a fast-growing community that includes:

- AI researchers and data scientists
- Enterprises leveraging AI for innovation and scale
- Developers building applications with advanced machine learning capabilities
- Educators and learners advancing AI knowledge

Our open-source tools and libraries are among the most widely used in the industry, powering innovation across multiple sectors.

---

## Get Connected

- Explore our AI models, datasets, and apps: [https://huggingface.co](https://huggingface.co)
- Join the conversation on our forums: [https://discuss.huggingface.co](https://discuss.huggingface.co)
- Follow us on Twitter: [https://twitter.com/huggingface](https://twitter.com/huggingface)
- Connect on LinkedIn: [https://www.linkedin.com/company/huggingface/](https://www.linkedin.com/company/huggingface/)
- Explore our code on GitHub: [https://github.com/huggingface](https://github.com/huggingface)

---

Hugging Face — Building the future of AI through open collaboration and innovation. Join us and be part of the AI community shaping tomorrow.